<a href="https://colab.research.google.com/github/vitthal-bhandari/Homophobia-Transphobia-Detection/blob/master/Baselines/Code_Mixed_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initializing environment

In [1]:
#check gpu usage
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Apr  6 12:41:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#check ram usage
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install datasets
!pip install transformers
!sudo apt-get install git-lfs
!pip3 install torch==1.10.2+cu102 torchvision==0.11.3+cu102 torchaudio===0.10.2+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 136 kB 56.2 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 212 kB 57.5 MB/s 
     |████████████████████████████████| 1.1 MB 58.1 MB/s 
     |████████████████████████████████| 127 kB 63.7 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 271 kB 59.2 MB/s 
     |████████████████████████████████| 144 kB 62.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 3.8 MB 5.1 MB/s 


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
import tensorflow as tf
import pandas as pd
import numpy as np
import torch

# Reading pre-processed data

In [5]:
eng_tam_train_dataset = load_dataset("csv", data_files="eng_tam_augmented_data.csv")
eng_tam_train = eng_tam_train_dataset["train"]

eng_tam_dev_dataset = load_dataset("csv", data_files="eng_tam_dev.csv")
eng_tam_dev = eng_tam_dev_dataset["train"]

eng_tam_test_dataset = load_dataset("csv", data_files="eng_tam_test.csv")
eng_tam_test = eng_tam_test_dataset["train"]

print(len(eng_tam_train), len(eng_tam_dev), len(eng_tam_test))

Using custom data configuration default-e03dc4c60aa10540


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e03dc4c60aa10540/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-f789361e95832ac7


...

Using custom data configuration default-79dc036ed29ab2cc


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-79dc036ed29ab2cc/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

3861 965 1207


# Declaring model name

In [6]:
model_ckpt = "bert-base-multilingual-cased"

# Tokenization

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [8]:
# Define tokenizer function

def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True)

In [9]:
# Ensuring there are no empty records so that tokenization doesn't fail

eng_tam_train = eng_tam_train.filter(lambda example: example['text'] is not None)
eng_tam_dev = eng_tam_dev.filter(lambda example: example['text'] is not None)
eng_tam_test = eng_tam_test.filter(lambda example: example['text'] is not None)

print( len(eng_tam_train), len(eng_tam_dev), len(eng_tam_test) )

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

3861 965 1207


In [10]:
# Apply tokenizer across all splits in the corpus

eng_tam_train_encoded = eng_tam_train.map(tokenize, batched=True, batch_size=None)
eng_tam_dev_encoded = eng_tam_dev.map(tokenize, batched=True, batch_size=None)
eng_tam_test_encoded = eng_tam_test.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
print(eng_tam_train_encoded.column_names)

['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']


# Feature Extraction

In [12]:
from transformers import AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

print(device)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda


In [13]:
def extract_hidden_states(batch):
  
  #Place model inputs on GPU
  inputs = {k:v.to(device) for k, v in batch.items() 
  if k in tokenizer.model_input_names}

  #Extract last hidden states
  with torch.no_grad():
    last_hidden_state = model(**inputs).last_hidden_state
  
  #Return vector for [CLS] token
  return {"hidden_state": last_hidden_state[:, 0].cpu().numpy()}

In [14]:
eng_tam_train_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
eng_tam_dev_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
eng_tam_test_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [15]:
eng_tam_train_hidden = eng_tam_train_encoded.map(extract_hidden_states, batched=True, batch_size=128)
eng_tam_dev_hidden = eng_tam_dev_encoded.map(extract_hidden_states, batched=True, batch_size=128)
eng_tam_test_hidden = eng_tam_test_encoded.map(extract_hidden_states, batched=True, batch_size=128)

  0%|          | 0/31 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [16]:
eng_tam_train_hidden.column_names

['label',
 'text',
 'input_ids',
 'token_type_ids',
 'attention_mask',
 'hidden_state']

In [17]:
X_eng_tam_train = np.array(eng_tam_train_hidden['hidden_state'])
X_eng_tam_dev = np.array(eng_tam_dev_hidden['hidden_state'])
X_eng_tam_test = np.array(eng_tam_test_hidden['hidden_state'])

Y_eng_tam_train = np.array(eng_tam_train_hidden['label'])
Y_eng_tam_dev = np.array(eng_tam_dev_hidden['label'])
Y_eng_tam_test = np.array(eng_tam_test_hidden['label'])

print( X_eng_tam_train.shape, X_eng_tam_dev.shape, X_eng_tam_test.shape, Y_eng_tam_train.shape, Y_eng_tam_dev.shape, Y_eng_tam_test.shape)

(3861, 768) (965, 768) (1207, 768) (3861,) (965,) (1207,)


# Training an LR classifier

In [18]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter = 3000)
lr_clf.fit(X_eng_tam_train, Y_eng_tam_train)

LogisticRegression(max_iter=3000)

In [19]:
lr_clf.score(X_eng_tam_dev, Y_eng_tam_dev)

0.8880829015544042

In [20]:
#Evaluating a dummy classifier for comparison which always returns the majority class

from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_eng_tam_train, Y_eng_tam_train)
dummy_clf.score(X_eng_tam_dev, Y_eng_tam_dev)

0.8922279792746114

In [21]:
Y_eng_tam_preds = lr_clf.predict(X_eng_tam_dev)

In [22]:
from sklearn.metrics import classification_report

target_names = ['homophobic', 'Non-anti LGBTQIA+ content', 'transphobic']
print(classification_report(Y_eng_tam_dev, Y_eng_tam_preds, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.28      0.14      0.18        66
Non-anti LGBTQIA+ content       0.91      0.97      0.94       861
              transphobic       0.65      0.29      0.40        38

                 accuracy                           0.89       965
                macro avg       0.61      0.47      0.51       965
             weighted avg       0.86      0.89      0.87       965

